### Optimize Parquet files

In [ ]:
# Create Spark Session

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Optimize Parquet Files") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

spark

In [ ]:
# Function to generate random data between 0-7
import random

def generate_data(cnt):
    _lst = []
    for i in range(0, cnt):
        num = random.choice(range(0,11))
        _lst.append([num])
    return _lst

In [ ]:
# Lets create a simple Python decorator - {get_time} to get the execution timings
# If you dont know about Python decorators - check out : https://www.geeksforgeeks.org/decorators-in-python/
import time

def get_time(func):
    def inner_get_time() -> str:
        start_time = time.time()
        func()
        end_time = time.time()
        return (f"Execution time: {(end_time - start_time)*1000} ms")
    print(inner_get_time())

In [ ]:
# Genearte dataframe with 10M random numbers
_data = generate_data(10000000)
_schema = "values int"

# Create dataframe
df = spark.createDataFrame(data = _data, schema=_schema)

In [ ]:
# Write data in Parquet format

@get_time
def x(): df.write.format("parquet").save("dataset/num/1/num.parquet")

In [ ]:
# Sort the data
from pyspark.sql.functions import col, asc
df_fixed = df.orderBy(col("values").asc())

In [ ]:
# Write in Parquet format
@get_time
def x(): df_fixed.write.format("parquet").save("dataset/num/2/num.parquet")

In [ ]:
# Read the data from location 1 (non-optimized)
@get_time
def x():
    df = spark.read.parquet("dataset/num/1/num.parquet")
    df.write.format("noop").mode("overwrite").save()

In [ ]:
# Read the data from location 2 (optimized)
@get_time
def x():
    df = spark.read.parquet("dataset/num/2/num.parquet")
    df.write.format("noop").mode("overwrite").save()